# Analysis of the results

In [67]:
import json
import pandas as pd
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000

In [68]:
dfs = {}
for i in ["05", "10", "15"]:
    df = pd.read_csv("stats-{:s}.csv".format(i))
    df["instance"] = df.apply(lambda row: row["instance"].split(".")[0], axis=1)
    # rename ILP
    df["solver"] = df.apply(lambda row: "ILP" if row["solver"] == "ILP Solver (global):predictor (Python)" else row["solver"], axis=1)
    # merge to separate columns by instance
    bap = df[df["solver"] == "BAP"]            
    ilp = df[df["solver"] == "ILP"]
    df = pd.merge(bap, ilp, on="instance",suffixes=["_bap", "_ilp"])
    
    dfs[i] = df

In [57]:
dfs["05"]

,instance,solver_bap,time_bap,objective-reported_bap,objective_bap,length_bap,metadata_bap,solver_ilp,time_ilp,objective-reported_ilp,objective_ilp,length_ilp,metadata_ilp
0,IN_000,BAP,22,-1.000000,0.000000,0,"{'number_of_nodes': '0', 'time_masters_init': ...",ILP,1,-1.000000,0.000000,0,{'objective': '-1'}
1,IN_001,BAP,78,1.146634,1.146634,60,"{'objective': '1.1466338951899544', 'number_of...",ILP,4,1.146634,1.146634,60,{'objective': '1.1466338951899542'}
2,IN_002,BAP,378,1.028355,1.028355,96,"{'objective': '1.0283551789016403', 'number_of...",ILP,5,1.028355,1.028355,96,{'objective': '1.0283551789016392'}
3,IN_003,BAP,80,0.633783,0.633783,133,"{'objective': '0.6337833695674279', 'number_of...",ILP,22,0.633783,0.633783,133,{'objective': '0.6337833695674275'}
4,IN_004,BAP,116,1.018899,1.018899,71,"{'objective': '1.0188986965767821', 'number_of...",ILP,30,1.018899,1.018899,71,{'objective': '1.0188986965767812'}
5,IN_005,BAP,462,2.003834,2.003834,59,"{'objective': '2.003834037927014', 'number_of_...",ILP,2,2.003834,2.003834,59,{'objective': '2.003834037927014'}
6,IN_006,BAP,177,1.315118,1.315118,84,"{'objective': '1.3151180542859628', 'number_of...",ILP,2,1.315118,1.315118,84,{'objective': '1.3151180542859628'}
7,IN_007,BAP,7,-1.000000,0.000000,0,"{'number_of_nodes': '0', 'time_masters_init': ...",ILP,1,-1.000000,0.000000,0,{'objective': '-1'}
8,IN_008,BAP,108,0.535914,0.535914,90,"{'objective': '0.535913511811783', 'number_of_...",ILP,31,0.535914,0.535914,90,{'objective': '0.5359135118117826'}
9,IN_009,BAP,209,1.785361,1.785361,83,"{'objective': '1.7853614548172831', 'number_of...",ILP,3,1.785361,1.785361,83,{'objective': '1.7853614548172831'}


In [76]:
for size in ["05", "10", "15"]:
    print()
    print(size)
    
    df = dfs[size]
    
    print("Overall performance:")
    for method in ["ilp", "bap"]:
        avg = df["time_" + method].mean() / 1000 # to s
        std = df["time_" + method].std() / 1000
        avg_nodes = df["metadata_bap"]
        print("time {:s} ${:4.2f} \pm {:4.2f}$".format(method, avg, std))    
                    
    # ILP was faster
    df_cur = df[df["time_ilp"] < df["time_bap"]]
    print(" - ilp was faster {:d} times".format(df_cur["instance"].count()))                
    for method in ["ilp", "bap"]:
        avg = df_cur["time_" + method].mean() / 1000
        std = df_cur["time_" + method].std() / 1000
        print("   time {:s} ${:4.2f} \\pm {:4.2f}$".format(method, avg, std))    
    bap_nodes_avg = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).mean()
    bap_nodes_min = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).min()
    bap_nodes_max = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).max()
    print("   bap nodes avg", bap_nodes_avg, "min", bap_nodes_min, "max", bap_nodes_max)
        
    
    df_cur = df[df["time_ilp"] > df["time_bap"]]
    print(" - bap was faster {:d} times".format(df_cur["instance"].count()))    
    for method in ["ilp", "bap"]:
        avg = df_cur["time_" + method].mean() / 1000
        std = df_cur["time_" + method].std() / 1000
        print("   time {:s} ${:4.2f} \\pm {:4.2f}$".format(method, avg, std))    
    bap_nodes_avg = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).mean()
    bap_nodes_min = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).min()
    bap_nodes_max = df_cur["metadata_bap"].apply(lambda row: int(json.loads(row)["number_of_nodes"])).max()
    print("   bap nodes avg", bap_nodes_avg, "min", bap_nodes_min, "max", bap_nodes_max)


05
Overall performance:
time ilp $0.02 \pm 0.03$
time bap $0.12 \pm 0.16$
 - ilp was faster 85 times
   time ilp $0.01 \pm 0.02$
   time bap $0.13 \pm 0.17$
   bap nodes avg 2.835294117647059 min 0 max 34
 - bap was faster 11 times
   time ilp $0.09 \pm 0.03$
   time bap $0.05 \pm 0.02$
   bap nodes avg 0.9090909090909091 min 0 max 1

10
Overall performance:
time ilp $1.31 \pm 2.48$
time bap $3.14 \pm 6.06$
 - ilp was faster 67 times
   time ilp $0.33 \pm 0.47$
   time bap $4.29 \pm 7.07$
   bap nodes avg 24.19402985074627 min 0 max 190
 - bap was faster 32 times
   time ilp $3.35 \pm 3.56$
   time bap $0.73 \pm 0.77$
   bap nodes avg 3.34375 min 1 max 40

15
Overall performance:
time ilp $36.75 \pm 96.28$
time bap $56.41 \pm 117.94$
 - ilp was faster 67 times
   time ilp $5.57 \pm 10.78$
   time bap $82.07 \pm 137.18$
   bap nodes avg 348.82089552238807 min 1 max 3796
 - bap was faster 33 times
   time ilp $100.05 \pm 149.24$
   time bap $4.32 \pm 6.53$
   bap nodes avg 12.2424242424

In [84]:
df = dfs["05"]
print(df[df["time_bap"] > 60]["instance"].count())
print(df[df["time_ilp"] > 60]["instance"].count())


59
12


,instance,time_bap,time_ilp
0,IN_000,22,1
1,IN_001,78,4
2,IN_002,378,5
3,IN_003,80,22
4,IN_004,116,30
5,IN_005,462,2
6,IN_006,177,2
7,IN_007,7,1
8,IN_008,108,31
9,IN_009,209,3
